In [1]:
# Configurations

model_cache_dir = "../model"
model_name = "Helsinki-NLP/opus-mt-ja-en"

dataset = "ryo0634/bsd_ja_en"
dataset_cache_dir = "../data"

In [2]:
# Load model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=model_cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, cache_dir=model_cache_dir)

/opt/anaconda3/envs/structured_document/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/structured_document/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [3]:
# Load datasets
from datasets import load_dataset, concatenate_datasets

ds = load_dataset(dataset, cache_dir=dataset_cache_dir)

In [4]:
# Merge datasets into one, remove unnecessary columns
train = ds["train"]
test = ds["test"]
validation = ds["validation"]

ds_merged = concatenate_datasets([train, test, validation])
ds_merged = ds_merged.remove_columns(['id', 'tag', 'title', 'original_language', 'no', 'en_speaker', 'ja_speaker'])

In [8]:
from datasets import Dataset

def tokenize_ja(example):
    return tokenizer(example["ja_sentence"], truncation=True, max_length=512, padding="max_length", return_tensors="pt")

def tokenize_prepare(dataset: Dataset) -> Dataset:
  tokenized_dataset = dataset.map(tokenize_ja, batched=True)
  tokenized_dataset.set_format("torch")
  return tokenized_dataset

In [9]:
ds_tokenized = tokenize_prepare(ds_merged)

Map: 100%|██████████| 24171/24171 [00:08<00:00, 2753.63 examples/s]


In [10]:
def generate_translation(batch):
    outputs = model.generate(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
    batch["translations"] = [tokenizer.decode(output) for output in outputs]
    return batch

translated_dataset = ds_tokenized.map(generate_translation, batched=True, batch_size=16)

Map:   0%|          | 32/24171 [01:11<15:03:19,  2.25s/ examples]
Exception ignored in: <function Dataset.__del__ at 0x121b856c0>
Traceback (most recent call last):
  File "/opt/anaconda3/envs/structured_document/lib/python3.11/site-packages/datasets/arrow_dataset.py", line 1421, in __del__
    def __del__(self):

KeyboardInterrupt: 


KeyboardInterrupt: 